# Notes
## Project Folder Structure
The input data for this project is too large to store with the github repo. Therefore, the files need to be added manually. The raw image files can be found here:

[https://www.kaggle.com/c/state-farm-distracted-driver-detection/data](https://www.kaggle.com/c/state-farm-distracted-driver-detection/data) (Accessed on 3/21/2020)

You can download the raw input files as a zip file. It will contain the following:


* driver_imgs_list.csv (This file in maintained in the repo)
* sample_submission.csv (This file is maintained in the repo)
* imgs folder, with test and train subfolders

This **imgs** folder is the part of the input which is not maintained in the repo. Therefore, for this notebook to function the data in the **imgs** 

--Working

----imgs

------raw
  
--------train

--------test

The train and test folders come directly from the kaggle dataset. Subsequent folders will be generated by this notebook.

## Scope
This notebook is to prepare the images in the distracted driver dataset locally, for upload to S3.

This notebook will create two versions of the data. The first will be a LST mapping file, with the image files in JPEG format. The second will be a RECORDIO format.

A Complete dataset and Sample dataset will be created and uploaded.

## Method

This notebook splits the training and validation set manually by driver, not by random selection.

# Import

Clone MXNET to get im2rec tool

In [82]:
#!git clone https://github.com/apache/incubator-mxnet.git

## Install

In [83]:
!pip install checksumdir
!pip install opencv-python mxnet

## Library / Packages

In [84]:
import pandas as pd
import random 
import os
import shutil
import boto3

import hashlib
from checksumdir import dirhash

from filecmp import dircmp

#Settings
seed = 5590

## Data

In [85]:
url = "https://raw.githubusercontent.com/DSBA-6190-Final-Project-Team/DSBA-6190_Final-Project/master/wine_predict/data/driver_imgs_list.csv"
path_file = 'data/driver_imgs_list.csv'

df_driver_index = pd.read_csv(path_file) 

# EDA

In [86]:
df_driver_index.shape

(22424, 3)

In [87]:
df_driver_index.head(5)

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [88]:
df_driver_index.columns

Index(['subject', 'classname', 'img'], dtype='object')

The following lists each unique driver along with the number of different classname and images are associated with each. Note the number of classnames is not unique, so the images an classnames have equal frequencies.

In [89]:
drivers_gb = df_driver_index.groupby(['subject'])
drivers_gb.count()

,classname,img
subject,,
p002,725,725
p012,823,823
p014,876,876
p015,875,875
p016,1078,1078
p021,1237,1237
p022,1233,1233
p024,1226,1226
p026,1196,1196


We will set the training/validation split ratio.

In [90]:
train_val_split = 0.2

To split the data into training and validation sets, we'll need a unique index of drivers. We don't need the frequency counts.

We will create a shuffled list of unique drivers.


In [91]:
drivers_unique = drivers_gb.groups.keys()
drivers_unique = list(drivers_unique)

random.Random(seed).shuffle(drivers_unique)
print(drivers_unique)

['p049', 'p022', 'p012', 'p075', 'p045', 'p072', 'p026', 'p042', 'p015', 'p039', 'p047', 'p081', 'p052', 'p014', 'p021', 'p035', 'p061', 'p064', 'p066', 'p056', 'p002', 'p041', 'p024', 'p016', 'p051', 'p050']


We'll set the list of drivers in the train set and the validation set.

In [92]:
num_drivers_val = round(len(drivers_unique)*train_val_split)
#print(num_drivers_val)
num_drivers_train = len(drivers_unique) - num_drivers_val
#print(num_drivers_train)

In [93]:
drivers_val = drivers_unique[:num_drivers_val]
print(drivers_val)

['p049', 'p022', 'p012', 'p075', 'p045']


In [94]:
drivers_train = drivers_unique[-num_drivers_train:]
print(drivers_train)

['p072', 'p026', 'p042', 'p015', 'p039', 'p047', 'p081', 'p052', 'p014', 'p021', 'p035', 'p061', 'p064', 'p066', 'p056', 'p002', 'p041', 'p024', 'p016', 'p051', 'p050']


# Training and Validation Image Lists
We'll now create two lists, one list of every image file name associated with the trainging set, another list associated with the validation set. 

We will use these lists to filter the overall lst mapping file.

In [95]:
df_images_val = df_driver_index[df_driver_index['subject'].isin(drivers_val)]
df_images_train = df_driver_index[~df_driver_index['subject'].isin(drivers_val)]

In [96]:
print(df_images_train.shape)
df_images_train.head()

(17819, 3)


,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [97]:
print(df_images_val.shape)
df_images_val.head()

(4605, 3)


,subject,classname,img
725,p012,c0,img_10206.jpg
726,p012,c0,img_27079.jpg
727,p012,c0,img_50749.jpg
728,p012,c0,img_97089.jpg
729,p012,c0,img_37741.jpg


# Move Images
We need to create two folders images, one for training, and one for validation. We will leave the complete folder of images alone, in case we need to go back and extract information from that folder

## Execute Copy
The following function takes the training or validaiton dataframe and iterates through each row. It then copies each file from the overall data to the respective folder. 

In [98]:
def copy_imgs_driver(status, df):
    prefix_subfolder = "imgs"
    prefix_src = "raw/train"
    prefix_dst = "train-split_driver"
    #prefix_status = os.path.join("imgs",folder_name)

    for index, row in df.iterrows():
        path_src = os.path.join(prefix_subfolder,
                                prefix_src,
                                row['classname'], 
                                row['img']).replace('\\','/')
             
       
        path_dst = os.path.join(prefix_subfolder,
                                prefix_dst,
                                status,
                                row['classname'],
                                row['img']).replace('\\','/')
        
        #print("Source File:\t\t{}".format(path_src))
        #print("Destination File:\t{}".format(path_dst))
    
        
        if not os.path.exists(path_dst):
            # Verify Directory Exists. Create if Not 
            os.makedirs(os.path.dirname(path_dst), exist_ok=True)
            
            shutil.copy(path_src, path_dst)
            #print("Loop Start")
            #print("Source File:\t\t{}".format(path_src))
            #print("Destination File:\t{}".format(path_dst))
            #print()
        #else:
            #print("File Exists. No Copy Made.")
            

We will create a dictionary containing the status and associated dataframes. This will be looped over.

In [99]:
dict_image_df = {
    "train": df_images_train,
    "validation" : df_images_val,
    "dummy": df_images_val
}

In [100]:
for key in dict_image_df:
    copy_imgs_driver(key, dict_image_df[key])

# Verify Folder Size

We will now verify the size of the resulting folders.

In [101]:
# Function calculates the number of files in a directory, recursively
def num_files(dir_scan):
    cpt = sum([len(files) for r, d, files in os.walk(dir_scan)])
    print("Directory Analyzed:\t{}".format(dir_scan))
    print("\t\t\tThere are {} image files in the directory.".format(cpt))
    print()

In [102]:
path_current = "D:\\Notebooks\\dsba_6190\\team_project\\image_classification_preprocessing"
prefix_split = "imgs\\train-split_driver"
prefix_train = "train"
prefix_val = "validation"

# Training
dir_train = os.path.join(path_current, prefix_split, prefix_train)
num_files(dir_train)

# Validation
dir_validation =  os.path.join(path_current, prefix_split, prefix_val)
num_files(dir_validation)

Directory Analyzed:	D:\Notebooks\dsba_6190\team_project\image_classification_preprocessing\imgs\train-split_driver\train
			There are 17819 image files in the directory.

Directory Analyzed:	D:\Notebooks\dsba_6190\team_project\image_classification_preprocessing\imgs\train-split_driver\validation
			There are 4605 image files in the directory.



# Process Flow

Some of the downstream processing steps are computationally heavy. Therefore, I'm going to put in some guardrails in order to avoid running these code chunks, without manually commenting them out once run. 

To check if any of the input files have changed, which would then require reprocessing the images, we will use Hash checksum values to document the state of the inputs. We will check against this state to determine if we need to reprocess the data. 

To do this, we will have to convert the processing steps into callable functions.

## im2rec Function Calls
The following functions use the im2rec.py tool to process the image inputs.

[https://mxnet.apache.org/api/faq/recordio](https://mxnet.apache.org/api/faq/recordio) (Accessed on 3/20/2020)

### LST File Generation
This function performs the first step, creating LST mapping files.

In [ ]:
def im2rec_lst(status, status_drop_train, dataset):
    lst_file_name = dataset
    lst_file_path = os.path.join("model_input_files", status_drop_train, dataset)
    img_loc =  os.path.join("imgs", status, dataset)
    
    %run incubator-mxnet/tools/im2rec.py --list --recursive {lst_file_path} {img_loc} 
    
    return img_loc, lst_file_path

### Recordio Conversion
This function converts the images into a RECORDIO binary file format. In addition to converting images from raw image files to RECORDIO, we need to resize the images. In the Amazon Sagemaker Image Classification Hyperparameters documentation ([here](https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html)) there is a parameter called **image_shape**. This parameter requires a string with three numbers, comma seperated (i.e. "1,2,3").

The first value is **num_channels**. This is the number of channels our input images have. They are color images, so they have three channels (Red, Green, and Blue aka RGB). The second and third values are the heigth and width of the images, in pixels. Technically, the algorithm can accomadate images of any size, but if the images are too large, there may be memory constraints. It indicates typical dimensions are 244 x 244. Our images come are 640 x 480. By area this is 5x bigger. So, we need to resize the images.

The tool **im2rec.py** is capable of resizing images during the RECORDIO conversion. All we need to to is add the argument "**--resize #**" to the command line entry. The number in the command line argument is what the tool will resize the ***shortest edge*** of the input image. The larger edge will be resized to maintain the same relative dimensions. 

Therefore, we are going to resize the input images so that the resulting area is approximately equal to the area of a 244 x 244 image. The dimensions we are going to use is 210 x 280. This maintains the relative size of the input images, while have an area 98.7% that of the default image size. So, in the command line argument, the call will be **--resize 210**.

In [104]:
def im2rec_rec(img_loc, lst_file_path):
    
    %run incubator-mxnet/tools/im2rec.py  --resize 210 {lst_file_path} {img_loc} 

## Writing Hash Checksum
The following function generates the Hash of a directory (training or validation images) and writes the Hash in a CSV

In [105]:
def write_hash(status, file_path, file_dir):

    # Generate Hash
    dir_hash = dirhash(file_dir, 'sha256')
    
    # Write to CSV
    dict_dir_hash = {'hash': [dir_hash]}
    df_dir_hash = pd.DataFrame(dict_dir_hash)
    df_dir_hash.to_csv(file_path, index=False)
    print("Hash for {} images successfully written to CSV.".format(status))
    print()    

## Combine Processing and Writing Hash
Whenever we process the input data we need to write a new hash, and vice versa. Therefore, for simplicity in the final code, we will lump these actions together.


In [106]:
def im2rec_and_write(status, status_drop_train, dataset, file_name, file_path, file_dir):
        print("Generating LST File: {} status - {} dataset".format(status, dataset))
        
        img_loc, lst_file_path = im2rec_lst(status, status_drop_train, dataset)
        
        print("Starting generating REC file: {} status - {} dataset".format(status, dataset))
        
        im2rec_rec(img_loc, lst_file_path)
        
        print("REC File complete.")
        
        write_hash(status, file_path, file_dir)

## Process Flow Function
The following function will verify the hash of the current files matches the older record. If they match, no action is taken. If they do not match, the input data is processed, and a new hash is generated.

In [111]:
def verify_hash(dataset, status):
    
    # Verify dataset variable is correct.
    dataset_list = ["sample", "complete", "train", "validation", "dummy"]
    
    if dataset not in dataset_list:
        print("Error. Correct Dataset Type Not Entered.")
        print("Dataset must be either type sample or complete.")
        return
    
    print()
    print("Dataset: {}".format(dataset))
    print()
            
              
    # Generate File Name, Path and Directory for Hash
    status_drop_train = status.split("-")[1]
    file_name = "hash_{}.csv".format(dataset)
    file_path_hash = os.path.join("hash", status_drop_train, file_name)
    file_dir = os.path.join('imgs', status, dataset)
    
    # Print for Sanity Check
    print("Hash File Name: {}".format(file_name))
    print("Hash File Path: {}".format(file_path_hash))
    print("Image Set File Directory: {}".format(file_dir))
    print()
    
    # Generate Current Hash
    print('New Hash - Generating')
    hash_new = dirhash(file_dir, 'sha256')
    print('New Hash - Generated')
    
    if not os.path.exists(file_path_hash):
        print("Hash for {} images in {} do not exist.".format(dataset, status))
        print()
        im2rec_and_write(status, status_drop_train, dataset, 
                         file_name, file_path_hash, file_dir)
        return
    
    else:
        # Read Existing Hash
        df_hash_old = pd.read_csv(file_path_hash)
        hash_old = df_hash_old.iloc[0]['hash']
        
        if hash_old == hash_new:
            print("Hash for {} images are equal.".format(status))
            print("New Hash not generated. Processing not required.")
            print()
            return
        
        else:
            print("Hash for {} images are NOT equal.".format(status))
            print()
            im2rec_and_write(status, status_drop_train, dataset, 
                             file_name, file_path_hash, file_dir)

## Execute Process
Runnign the following code chuck will execute the process flow function develpoed above for both training and validation data.

In [113]:
driver_dataset_list = ["train", "validation", "dummy"]

status = "train-split_driver"

for dataset in driver_dataset_list:
    verify_hash(dataset, status)


Dataset: train

Hash File Name: hash_train.csv
Hash File Path: hash\split_driver\hash_train.csv
Image Set File Directory: imgs\train-split_driver\train

New Hash - Generating
New Hash - Generated
Hash for train images in train-split_driver do not exist.

Generating LST File: train-split_driver status - train dataset
c0 0
c1 1
c2 2
c3 3
c4 4
c5 5
c6 6
c7 7
c8 8
c9 9
Starting generating REC file: train-split_driver status - train dataset
Creating .rec file from D:\Notebooks\dsba_6190\team_project\image_classification_preprocessing\model_input_files\split_driver\train.lst in D:\Notebooks\dsba_6190\team_project\image_classification_preprocessing\model_input_files\split_driver
multiprocessing not available, fall back to single threaded encoding
time: 0.005982398986816406  count: 0
time: 6.443885564804077  count: 1000
time: 8.041106939315796  count: 2000
time: 6.454342603683472  count: 3000
time: 6.0480637550354  count: 4000
time: 7.386298656463623  count: 5000
time: 6.475639343261719  coun

Traceback (most recent call last):
  File "D:\Notebooks\dsba_6190\team_project\image_classification_preprocessing\incubator-mxnet\tools\im2rec.py", line 179, in image_encode
    img = cv2.imread(fullpath, args.color)
KeyboardInterrupt


TypeError: object of type 'NoneType' has no len()

REC File complete.
Hash for train-split_driver images successfully written to CSV.


Dataset: validation

Hash File Name: hash_validation.csv
Hash File Path: hash\split_driver\hash_validation.csv
Image Set File Directory: imgs\train-split_driver\validation

New Hash - Generating
New Hash - Generated
Hash for validation images in train-split_driver do not exist.

Generating LST File: train-split_driver status - validation dataset
c0 0
c1 1
c2 2
c3 3
c4 4
c5 5
c6 6
c7 7
c8 8
c9 9
Starting generating REC file: train-split_driver status - validation dataset
Creating .rec file from D:\Notebooks\dsba_6190\team_project\image_classification_preprocessing\model_input_files\split_driver\validation.lst in D:\Notebooks\dsba_6190\team_project\image_classification_preprocessing\model_input_files\split_driver
multiprocessing not available, fall back to single threaded encoding
time: 0.011969327926635742  count: 0
time: 9.530853509902954  count: 1000
time: 7.656746864318848  count: 2000
time: 8.242354

Traceback (most recent call last):
  File "D:\Notebooks\dsba_6190\team_project\image_classification_preprocessing\incubator-mxnet\tools\im2rec.py", line 179, in image_encode
    img = cv2.imread(fullpath, args.color)
KeyboardInterrupt


TypeError: object of type 'NoneType' has no len()

REC File complete.
Hash for train-split_driver images successfully written to CSV.


Dataset: dummy

Hash File Name: hash_dummy.csv
Hash File Path: hash\split_driver\hash_dummy.csv
Image Set File Directory: imgs\train-split_driver\dummy

New Hash - Generating


KeyboardInterrupt: 

# S3 Upload

## Establish AWS Settings

In [59]:
profile = 'dsba_6190_proj_4'
region = 'us-east-1'
bucket = 'dsba-6190-final-team-project'
prefix = "channels_rec"
split_method = "split_driver"

In [40]:
session = boto3.session.Session(profile_name = profile,
                               region_name = region)

s3_resource = session.resource('s3')

## Upload
We now upload the Recordio format files to S3. The Amazon Sagemaker Algorithm requires a specific folder format, with the training and validation data as seperate subfolders of the same directory, with the names train and validation, respectively.

In [54]:
def upload_to_s3(status, dataset):
    # Establish Dynamic File and Path Names
    folder = "model_input_files"
    file_name_local = status + "_split_driver_"  + dataset + ".rec"
    file_name_s3 = status +  ".rec"
    
    # Define Boto3 Resource Inputs
    file_for_upload =  os.path.join(folder, split_method, file_name_local).replace('\\','/')
    s3_key = os.path.join(prefix, split_method, dataset, status, file_name_s3).replace('\\','/')
    
    print("File for Upload:\t{}".format(file_for_upload))
    print("s3 Key:\t\t\t{}".format(s3_key))
    
    # Upload
    s3_resource.Bucket(bucket).upload_file(Filename = file_for_upload, Key = s3_key)

### Complete Dataset

#### Training

In [55]:
status = status_list[0]
dataset = dataset_list[0]

#upload_to_s3(status, dataset)

File for Upload:	model_input_files/split_driver/train_subset_complete.rec
s3 Key:			channels/split_driver/complete/train/train.rec


#### Validation

In [56]:
status = status_list[1]
dataset = dataset_list[0]

#upload_to_s3(status, dataset)

File for Upload:	model_input_files/split_driver/validation_subset_complete.rec
s3 Key:			channels/split_driver/complete/validation/validation.rec


### Sample Dataset

#### Training

In [57]:
status = status_list[0]
dataset = dataset_list[1]

#upload_to_s3(status, dataset)

File for Upload:	model_input_files/split_driver/train_subset_sample.rec
s3 Key:			channels/split_driver/sample/train/train.rec


#### Validation

In [58]:
status = status_list[1]
dataset = dataset_list[1]

#upload_to_s3(status, dataset)

File for Upload:	model_input_files/split_driver/validation_subset_sample.rec
s3 Key:			channels/split_driver/sample/validation/validation.rec
